In [16]:
import numpy as np 
from matplotlib import pyplot as plt 
import pandas as pd

In [17]:
data =  pd.read_csv('all_data_compiled.csv')
print(data)

      index_compiled  index  class  B_B_FLUX_AP  B_B_FLUX_AP_HI  \
0                  0     89    1.0     0.000174        0.000182   
1                  1    370    1.0     0.000462        0.000477   
2                  2    645    0.0     0.000003        0.000003   
3                  3    275    1.0     0.000002        0.000002   
4                  4    474    0.0     0.000002        0.000002   
...              ...    ...    ...          ...             ...   
1466            1466    221    1.0     0.000002        0.000002   
1467            1467    864    0.0     0.000000        0.000000   
1468            1468    777    0.0     0.000080        0.000084   
1469            1469    831    0.0     0.000000        0.000000   
1470            1470    374    1.0     0.001739        0.001878   

      B_B_FLUX_AP_LO  B_M_FLUX_AP  B_M_FLUX_AP_HI  B_M_FLUX_AP_LO  \
0           0.000166     0.000569        0.000616        0.000526   
1           0.000444     0.005532        0.005737        

In [19]:
print(data.shape)
x = data.drop(['index_compiled', 'index', 'class'],axis=1)
print(x.shape)
y = data['class']
y.shape

(1471, 364)
(1471, 361)


(1471,)

In [20]:
x_np =  x.to_numpy()
y_np =  y.to_numpy()
x_train = x_np[:1000,:]
y_train = y_np[:1000]
print(x_train.shape)
print(y_train.shape)

(1000, 361)
(1000,)


In [21]:
def split_data(x,y,s):   
    split = 0.7 
    split_no = int(split*len(y))
    x_train = x[:split_no, :]
    y_train = y[:split_no]
    x_test = x[split_no:, :]
    y_test = y[split_no:]
    return ((x_train,y_train) , (x_test ,y_test))

(x_train,y_train),(x_test,y_test) = split_data(x_np, y_np, 0.8)
print(x_train.shape ,y_train.shape)
print(x_test.shape ,y_test.shape)

(1029, 361) (1029,)
(442, 361) (442,)


# Neural Netork starts here

In [22]:
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

In [53]:
one_hot_y_train =  to_categorical(y_train)
one_hot_y_test =  to_categorical(y_test)

In [54]:
def model_gen(shape):

    inputs =  keras.Input(shape=(361,))
    dense =  layers.Dense(64, activation='relu')
    x = dense(inputs)
    x =  layers.BatchNormalization(axis=-1)(x)
    #x = layers.Dropout(0.3)(x)
    for s in shape:
        x = layers.Dense(64, activation='relu')(x)
    
    outputs = layers.Dense(2 , activation='sigmoid')(x)
    model = keras.Model(inputs=inputs , outputs=outputs , name='trial_model')
    model.compile(
        loss = "categorical_crossentropy",
        optimizer = keras.optimizers.Adam(),
        metrics = ["accuracy"],
    )
    return model
model = model_gen([64,64,32,8])
history = model.fit(x_train, one_hot_y_train, batch_size=128, epochs=40, validation_split=0.1)

Epoch 1/40
8/8 [==============================] - 1s 33ms/step - loss: 0.6239 - accuracy: 0.6836 - val_loss: 0.6643 - val_accuracy: 0.6408
Epoch 2/40
8/8 [==============================] - 0s 6ms/step - loss: 0.4855 - accuracy: 0.7765 - val_loss: 0.6267 - val_accuracy: 0.6796
Epoch 3/40
8/8 [==============================] - 0s 7ms/step - loss: 0.4194 - accuracy: 0.8132 - val_loss: 0.6031 - val_accuracy: 0.6990
Epoch 4/40
8/8 [==============================] - 0s 7ms/step - loss: 0.3677 - accuracy: 0.8488 - val_loss: 0.5974 - val_accuracy: 0.7184
Epoch 5/40
8/8 [==============================] - 0s 7ms/step - loss: 0.3337 - accuracy: 0.8542 - val_loss: 0.5836 - val_accuracy: 0.7087
Epoch 6/40
8/8 [==============================] - 0s 6ms/step - loss: 0.2945 - accuracy: 0.8737 - val_loss: 0.5761 - val_accuracy: 0.7476
Epoch 7/40
8/8 [==============================] - 0s 7ms/step - loss: 0.2684 - accuracy: 0.8898 - val_loss: 0.5659 - val_accuracy: 0.6990
Epoch 8/40
8/8 [=================

In [55]:
def acc_score(model , y_test , x_test):
    y_pred = model.predict(x_test)
    correct = 0
    total =  len(y_test)
    for y1,y2 in zip(y_pred,y_test):
        if(int(y1)==int(y2)):
            correct+=1
        else:
            continue
    print('correct prediction :' , correct)
    print('total prediction :' , total)
    print('score: ' , correct/total*100)
    
print('training data prediction')
acc_score(model, one_hot_y_train, x_train)
print('----------------------------------')
print('test data prediciton')
acc_score(model, y_test, x_test)

training data prediction


TypeError: only size-1 arrays can be converted to Python scalars

In [60]:
res = model.predict(x_test)
res = [np.argmax(r) for r in res]
print(res)

[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 

In [62]:
count = 0
for a,b in zip(res,y_test):
    if(int(a)==int(b)):
        count+=1
print(count , len(y_test))

369 442
